In [1]:
import pysubs2
import spacy
import pandas as pd
import numpy as np
import os.path
from collections import Counter

from spacy.tokens import Doc # to create empty Doc
from spacy.vocab import Vocab # to create empty Vocab


In [2]:
nlp_en = spacy.load('en')
nlp_fr = spacy.load('fr')

In [21]:
def load_data():
    if os.path.exists('bcbc_fr_parsed') and \
        os.path.exists('bcbc_en_parsed.bin'):
        return Doc(Vocab()).from_disk('bcbc_fr_parsed.bin'), \
                    Doc(Vocab()).from_disk('bcbc_en_parsed.bin')
    else:  
        subs_french = pysubs2.load('bon_french/Bon.Cop.Bad.Cop.2006.720p.BluRay.H264.AAC-RARBG.srt',
                            encoding='iso-8859-1')
        subs_eng = pysubs2.load('bon_eng/Bon.Cop.Bad.Cop.2006.720p.BluRay.x264-.YTS.AG.srt')
        french_subs = " ".join(line.text for line in subs_french)
        french_parsed = nlp_fr(french_subs)

        eng_subs = " ".join(line.text for line in subs_eng)
        eng_parsed = nlp_en(eng_subs)

        french_parsed.to_disk('bcbc_fr_parsed.bin')
        eng_parsed.to_disk('bcbc_en_parsed.bin')

        return french_subs, french_parsed, eng_subs, eng_parsed

In [33]:
french_subs, french_parsed, eng_subs, eng_parsed = load_data()

In [5]:
[x.tag_ for x in french_parsed]
# TODO: Get tokens from each script and align them somehow
#       Maybe they don't have to be aligned, just use indexing to get
#       "corresponding" token from other script
#       Keep track of current script and switch according to prob_matrix 
#       below, choose one monolingual script for starters

['PUNCT___',
 'PRON__Number=Sing|Person=3',
 'VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
 'ADP___',
 'DET__Definite=Ind|Gender=Masc|Number=Sing|PronType=Art',
 'ADJ__Number=Sing',
 'ADV___',
 'PRON__Number=Sing|Person=3',
 'VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
 'ADP___',
 'PROPN__Gender=Masc|Number=Sing',
 'PUNCT___',
 'PROPN__Gender=Masc|Number=Sing',
 'PUNCT___',
 'NOUN__Gender=Masc|Number=Sing',
 'PROPN___',
 'PUNCT___',
 'PROPN___',
 'PUNCT___',
 'ADV__PronType=Int',
 'PRON__Gender=Masc|Number=Sing|Person=3|PronType=Dem',
 'VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
 'PUNCT___',
 'PUNCT___',
 'NOUN__Number=Plur',
 'PUNCT___',
 'ADV__Polarity=Neg',
 'ADJ__Number=Sing',
 'PUNCT___',
 'ADV___',
 'ADV__Polarity=Neg',
 'ADJ__Number=Sing',
 'PUNCT___',
 'ADV__Polarity=Neg',
 'ADJ__Number=Sing',
 'PUNCT___',
 'ADV___',
 'PUNCT___',
 'PUNCT___',
 'PRON__PronType=Int',
 'PRON__Number=Sing|Person=3',
 'AUX__Mood=Ind|Number=Sing|P

In [6]:
bcbc_gs = pd.read_excel('BCBC_GS_annotator1.xlsx')

    eng frn
eng
frn 

In [7]:
bcbc_gs = bcbc_gs[bcbc_gs['Listened Language'] != 'French/English']
bcbc_gs = bcbc_gs[bcbc_gs['Listened Language'] != "None"]
bcbc_gs = bcbc_gs[bcbc_gs['Listened Language'].notnull()]
bcbc_gs['Listened Language'].value_counts()

French     7650
English    5803
Name: Listened Language, dtype: int64

In [8]:
def prob_matrix(tokens, langs):
    """
    tokens: a pandas series containing all the tokens
    """
    
    switches = {lang: {}  for lang in langs}
    counts = Counter(zip(tokens, tokens[1:]))

    for (x, y), c in counts.items():
        switches[x][y] = c / float(len(tokens) - 1)
        
    return switches

In [9]:
prob = prob_matrix(bcbc_gs['Listened Language'], \
                             set(bcbc_gs['Listened Language']))
prob

{'English': {'French': 0.019699672911091287, 'English': 0.4116116562592923},
 'French': {'French': 0.5489146595301814, 'English': 0.01977401129943503}}

In [63]:
fr_subs = french_subs.split(" ")
en_subs = eng_subs.split(" ")
len(en_subs), len(fr_subs)


In [64]:
def lang_switcher(corpora, langs, start_lang):
    last_lang = start_lang
    last_index = langs.index(start_lang)
    cs_script = cs_script = corpora[last_lang][0] + " "
    
    for index, word in enumerate(en_subs):
        r = np.random.uniform(low=0.0, high=1.0)
        last_lang = langs[last_index]
    
        # Check if we switch
        if r > prob[last_lang][last_lang]:
            # Switch and get next word from other corpus
            last_index = last_index ^ 1
            last_lang = langs[last_index]
        
        cs_script += corpora[last_lang][index] + " "

    return cs_script

In [67]:
corpora = {"English": en_subs, "French": fr_subs}
langs = ["English", "French"]

eng_cs = lang_switcher(corpora, langs, "English")
fr_cs = lang_switcher(corpora, langs, "French")

# TODO: Pretty print these to remove all the slashes since it looks terrible
#       Maybe separate into sentences?
#       Try/catch block for last line since corpora are different lengths
#       Find itertools function or something instead of using XOR for index

'- - On passe à un autre {\\i1}- on you\'re à Patrick. Patrick, bonsoir.\\N- I do for ça buddy?{\\i0} {\\i1}- Ah, heard pire, pas pire,\\Npas the Toronto team{\\i0}\\N{\\i1}might be sold to c\'est tu penses de rumour, toi, la rumeur I Toronto\\Nserait vendu accurate Houston? {\\i1}It Toronto à Houston?\\NOuais, non, the mouth{\\i0}\\N{\\i1}of my non, {\\i1}- La give rumeur,\\Nc\'est pas ça give tout, Patrick. cousin!{\\i0} The taxi mon beau-frère,\\Nil bartender, considéré... - Beau-frère... brother-in-law. cousin, belle-soeur, le cousin... Who cares le the euh... Mononcle, fils rumour is,\\Na du fils from beau-père, ça be sold to pas U.S. secondes! it\'s not y a Taranna, Montreal.{\\i0} canadienne, ça well, être Vancouver,\\Nça peut since{\\i0}\\N{\\i1}the lockout things have been Edmonton,\\NToronto {\\i1}- Lockout? What peut déménager,\\Nça, je vous le donne. lockout?{\\i0} Quand You know, Mr. lock-out... - Le that cheap Lock-out!\\NIl crap{\\i0}\\N{\\i1}like où, le mascots Quoi, {\